# Football data analysis - Team 4

# Introduction

...

# Data manipulation

In [20]:
import pandas as pd
import numpy as np
import os


Since the structure of the data folder is complex (some countries include multiple leagues, some not), we somehow need to iterate over all folders and all end files.

In [34]:
# We will save the root directory of the data
root_dir = "raw_data"  
file_paths = []

# Now we can walk through the directory structure
for dirpath, dirnames, filenames in os.walk(root_dir):

    # Check each file in the current directory
    for filename in filenames:

        # Filter for CSV files
        if filename.endswith(".csv"):
            file_path = os.path.join(dirpath, filename)

            # Keep track of all file paths
            file_paths.append(file_path)
            
            
# Let's look at the first file we found and see if we can load it
data = pd.read_csv(file_paths[0])
data.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,B1,26/07/2019,19:30,Genk,Kortrijk,2,1,H,0,1,...,2.53,-1.25,1.98,1.88,2.01,1.88,2.05,1.96,1.96,1.87
1,B1,27/07/2019,17:00,Cercle Brugge,Standard,0,2,A,0,0,...,2.24,0.50,1.88,1.98,1.91,1.99,1.95,2.02,1.89,1.94
2,B1,27/07/2019,19:00,St Truiden,Mouscron,0,1,A,0,1,...,2.10,-0.25,1.83,2.02,1.87,2.03,1.88,2.07,1.83,2.02
3,B1,27/07/2019,19:00,Waregem,Mechelen,0,2,A,0,1,...,2.28,0.25,1.72,2.07,1.75,2.17,1.80,2.28,1.72,2.15
4,B1,27/07/2019,19:30,Waasland-Beveren,Club Brugge,1,3,A,1,1,...,2.73,1.50,2.20,1.70,2.19,1.74,2.25,1.83,2.11,1.74


In [35]:
# Let's initialize an empty list to hold our dataframes
all_features_list = []
for file_path in file_paths:

    # Read the CSV file into a DataFrame
    temp_df = pd.read_csv(file_path)

    # Extract all feature names (column names)
    all_features = temp_df.columns.tolist()

    # Append to the list to compare across files
    all_features_list.append(all_features)
    print(f"File: {file_path}, All feature names: {all_features}")


File: raw_data\belgium\1\1920.csv, All feature names: ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA', 'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5', 'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA', 'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA', 'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA', 'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5', 'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA', 'AvgCAHH', 'AvgCAHA']
File: raw_data\belgium\1\2021.csv, 

Even between the first two countries (England and Belgium), we can see that the features differ: data from England includes a "Referee" column. This makes data transformation more difficult.